In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/amex-2019/test_preds_3.csv
/kaggle/input/amex-2019/val_0_full_enc_trx.csv
/kaggle/input/amex-2019/train_fold_0_h2o transform.csv
/kaggle/input/amex-2019/test_fold_2_h2o transform.csv
/kaggle/input/amex-2019/val_fold_1_h2o transform.csv
/kaggle/input/amex-2019/LGBM_oof_withtrans.npy
/kaggle/input/amex-2019/test_preds_4.csv
/kaggle/input/amex-2019/campaign_data.csv
/kaggle/input/amex-2019/test_preds_h2o.csv
/kaggle/input/amex-2019/train_fold_3_h2o transform.csv
/kaggle/input/amex-2019/coupon_item_mapping.csv
/kaggle/input/amex-2019/test_fold_0_h2o transform.csv
/kaggle/input/amex-2019/submission_fulltrain_ens.csv
/kaggle/input/amex-2019/submission_fulltrain_max.csv
/kaggle/input/amex-2019/train_fold_2_h2o transform.csv
/kaggle/input/amex-2019/test_fold_4_h2o transform.csv
/kaggle/input/amex-2019/test_preds_0 GPU.csv
/kaggle/input/amex-2019/val_fold_3_h2o transform.csv
/kaggle/input/amex-2019/item_data.csv
/kaggle/input/amex-2019/train_fold_1_h2o transform.csv
/kaggle/input/

In [2]:
Path ='../input/amex-2019/'
targetcol = 'redemption_status'
n_splits = 5

In [3]:
subm = pd.read_csv(Path+'sample_submission.csv')

In [4]:
test_h2o_0_gpu = pd.read_csv(Path+'test_preds_0 GPU.csv')[targetcol].values
test_h2o_0_cpu = pd.read_csv(Path+'test_preds_0_CPU.csv')[targetcol].values

test_h2o=[]
predictions= np.zeros(subm.shape[0])

test_h2o +=[test_h2o_0_gpu]
predictions += test_h2o_0_gpu / n_splits 

for i in range(1,5):
    cur_preds = pd.read_csv(Path+'test_preds_{0}.csv'.format(i))[targetcol].values
    predictions += cur_preds / n_splits 
    test_h2o += [cur_preds]

In [5]:
subm[targetcol] = predictions
subm.to_csv('submission_h2o_test_preds_ens.csv',index=False)
subm.head()

,id,redemption_status
0,3,0.350997
1,4,0.056062
2,5,0.049938
3,8,0.041309
4,10,0.041584


In [6]:
predictions_cpu= np.zeros(subm.shape[0])
predictions_cpu += test_h2o_0_cpu / n_splits 
#predictions cpu 1st fold
for i in range(1,5):
    cur_preds = test_h2o[i]
    predictions_cpu += cur_preds / n_splits 

In [7]:
subm[targetcol] = predictions_cpu
subm.to_csv('submission_h2o_test_preds_ens_cpu_0.csv',index=False)
subm.head()

,id,redemption_status
0,3,0.247817
1,4,0.051986
2,5,0.049339
3,8,0.041485
4,10,0.041748


In [8]:
#read full train preds csv
# ens_full_train = pd.read_csv(Path+'submission_fulltrain_ens.csv')[targetcol].values
ens_full_train = pd.read_csv(Path+'submission_multilevel_fulltrain_ens.csv')[targetcol].values


In [9]:
#ensemble of h2o preds and full train preds
predictions_ens = (predictions + ens_full_train) / 2

subm[targetcol] = predictions_ens
subm.to_csv('submission_ens_h2o_test_preds_full_train.csv',index=False)
subm.head()

,id,redemption_status
0,3,0.295338
1,4,0.037100
2,5,0.039205
3,8,0.022487
4,10,0.022698


In [10]:
#ensemble of 0.1 h2o preds and full train preds
predictions_ens = (0.1*predictions + ens_full_train) 

subm[targetcol] = predictions_ens
subm.to_csv('submission_ens_h2o_test_preds_full_train_0.1.csv',index=False)
subm.head()

,id,redemption_status
0,3,0.274779
1,4,0.023744
2,5,0.033465
3,8,0.007796
4,10,0.007971


In [11]:
#ensemble of 0.2 h2o preds and full train preds
predictions_ens = (0.2*predictions + ens_full_train) 

subm[targetcol] = predictions_ens
subm.to_csv('submission_ens_h2o_test_preds_full_train_0.2.csv',index=False)
subm.head()

,id,redemption_status
0,3,0.309879
1,4,0.029350
2,5,0.038459
3,8,0.011927
4,10,0.012129


In [12]:
predictions_weighted= np.zeros(subm.shape[0])
w = [0.424, 0.07, 0.43, 0.03, 0.05]
predictions_weighted += w[0] * test_h2o_0_gpu 
#predictions cpu 1st fold
for i in range(1,5):
    cur_preds = test_h2o[i]
    predictions_weighted += w[i] * cur_preds 

In [13]:
subm[targetcol] = predictions_weighted
subm.to_csv('submission_h2o_test_preds_ens_weighted.csv',index=False)
subm.head()

,id,redemption_status
0,3,0.400717
1,4,0.031613
2,5,0.018877
3,8,0.013473
4,10,0.013714
